In [1]:
from types import SimpleNamespace
from climpy.wrf.pp_wrf_like_tropomi_so2 import pp_wrf_like_tropomi_so2
from climpy.wrf.pp_wrf_like_tropomi_ch4 import pp_wrf_like_tropomi_ch4
from climpy.wrf.pp_wrf_like_tropomi_no2 import pp_wrf_like_tropomi_no2

%load_ext autoreload
%autoreload 2

from argparse import Namespace
from climpy.utils.tropomi_utils import SENTINEL_DATA_ROOT_PATH, configure_tropomi_credentials, \
    fetch_tropomi_from_wrf_folder, regrid_tropomi_on_wrf_grid_in_batch, derive_information_fraction, \
    add_dates_to_metadata, get_tropomi_configs
import xarray as xr
from pathlib import Path
import glob, os
from datetime import datetime
import pandas as pd


'''
The goal:
Given the folder with WRF output, download TROPOMI data and calculate corresponding TROPOMI-like diagnostics
'''
# Input args
aqaba_defaults = SimpleNamespace(
    grid_id='AQABA_d01',
    wrf_output_folder_path = '/scratch/osipovs/Data/AirQuality/THOFA/chem_100_v2025.0/',
    wrf_filter_dates = None
)
thofa_defaults = SimpleNamespace(
    grid_id='THOFA_d02',
    wrf_output_folder_path = '/scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/',
    # wrf_output_folder_path = '/scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_revised/',
    wrf_filter_dates = [datetime(2023, 6, 1), datetime(2023, 6, 25)]  # last wrfout file is not full. Tropomi-like diags break
)
ch4_settings, no2_settings, so2_settings, o3_settings = get_tropomi_configs()
thofa_ch4_config = SimpleNamespace(**{**vars(thofa_defaults), **vars(ch4_settings)})
thofa_no2_config = SimpleNamespace(**{**vars(thofa_defaults), **vars(no2_settings)})
thofa_so2_config = SimpleNamespace(**{**vars(thofa_defaults), **vars(so2_settings)})
thofa_o3_config = SimpleNamespace(**{**vars(thofa_defaults), **vars(o3_settings)})

config = thofa_ch4_config
config = thofa_no2_config
config = thofa_so2_config
config = thofa_o3_config

ERROR 1: PROJ: proj_create_from_database: Open of /home/osipovs/apps/mambaforge/envs/py310/share/proj failed


# Getting List of TROPOMI files, CDSE Approach

Alternative way to Download TROPOMI: https://gist.github.com/nicholasbalasus/008b34590fbc55b757fbd879bb64ccc0


In [2]:
config.meta_fp = config.wrf_output_folder_path + 'pp/tropomi_meta_{}.csv'.format(config.diag_key.lower())
if os.path.exists(config.meta_fp):
    print('Reading existing metadata from {}'.format(config.meta_fp))
    meta_df = pd.read_csv(config.meta_fp)
else:
    meta_df = fetch_tropomi_from_wrf_folder(config)#, wrf_date_format='%Y-%m-%d_%H:%M:%S')
    if config.wrf_filter_dates: meta_df = meta_df[meta_df['start_date'].between(config.wrf_filter_dates[0], config.wrf_filter_dates[1])]

    # Build the WRF post-processing list to derive TROPOMI-like diagnostics
    meta_df['wrfin_file'] = meta_df['start_date'].apply(lambda x: x.strftime('wrfout_d01_%Y-%m-%d_00_00_00'))  # name of the wrf output
    meta_df['wrfout_file'] = meta_df['start_date'].apply(lambda x: x.strftime('wrfout_d01_%Y-%m-%d_%H_%M_%S'))  # name for the pp-ed wrf file

    # Save list of files for processing
    print('Saving metadata to \n{}'.format(config.meta_fp))
    Path(config.meta_fp).parent.mkdir(parents=True, exist_ok=True)  # # 2. Extract the parent directory and create it
    meta_df.to_csv(config.meta_fp, index=False)#, header=['Name'])

display(meta_df[['Id', 'Name', 'S3Path', 'GeoFootprint']].head(3))
display(meta_df[['Name', 'start_date', 'wrfin_file' , 'wrfout_file']].head(3))

Reading existing metadata from /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_meta_o3.csv


,Id,Name,S3Path,GeoFootprint
0,29231ba1-f880-47f7-8929-6a100f851732,S5P_OFFL_L2__O3__PR_20230601T081351_20230601T0...,/eodata/Sentinel-5P/TROPOMI/L2__O3__PR/2023/06...,"{'type': 'Polygon', 'coordinates': [[[-92.4468..."
1,9fa7cc0c-b49b-4760-8637-dc16c01f266a,S5P_OFFL_L2__O3__PR_20230601T095521_20230601T1...,/eodata/Sentinel-5P/TROPOMI/L2__O3__PR/2023/06...,"{'type': 'Polygon', 'coordinates': [[[-117.821..."
2,657b5b55-3188-4c40-a0aa-84453b4c3132,S5P_OFFL_L2__O3__PR_20230602T075450_20230602T0...,/eodata/Sentinel-5P/TROPOMI/L2__O3__PR/2023/06...,"{'type': 'Polygon', 'coordinates': [[[-87.7257..."


,Name,start_date,wrfin_file,wrfout_file
0,S5P_OFFL_L2__O3__PR_20230601T081351_20230601T0...,2023-06-01 08:13:51,wrfout_d01_2023-06-01_00_00_00,wrfout_d01_2023-06-01_08_13_51
1,S5P_OFFL_L2__O3__PR_20230601T095521_20230601T1...,2023-06-01 09:55:21,wrfout_d01_2023-06-01_00_00_00,wrfout_d01_2023-06-01_09_55_21
2,S5P_OFFL_L2__O3__PR_20230602T075450_20230602T0...,2023-06-02 07:54:50,wrfout_d01_2023-06-02_00_00_00,wrfout_d01_2023-06-02_07_54_50


In [24]:
import eofetch

sentinel_data_root_path = SENTINEL_DATA_ROOT_PATH
display('Downloading TROPOMI files to: {}'.format(sentinel_data_root_path))
configure_tropomi_credentials()

for index, row in meta_df.iterrows():
    print('downloading {}'.format(row.Name))
    eofetch.download(row.Name, target_directory=sentinel_data_root_path)

print("Done")

'Downloading TROPOMI files to: /project/k10048/osipovs//Data/Copernicus/Sentinel-5P/'

downloading S5P_OFFL_L2__O3__PR_20230601T081351_20230601T095521_29183_03_020500_20230603T044544.nc
downloading S5P_OFFL_L2__O3__PR_20230601T095521_20230601T113651_29184_03_020500_20230603T053855.nc
downloading S5P_OFFL_L2__O3__PR_20230602T075450_20230602T093620_29197_03_020500_20230604T000109.nc
downloading S5P_OFFL_L2__O3__PR_20230602T093620_20230602T111750_29198_03_020500_20230604T015005.nc
downloading S5P_OFFL_L2__O3__PR_20230603T073550_20230603T091720_29211_03_020500_20230604T235322.nc
downloading S5P_OFFL_L2__O3__PR_20230603T091720_20230603T105850_29212_03_020500_20230605T012718.nc
downloading S5P_OFFL_L2__O3__PR_20230604T085819_20230604T103949_29226_03_020500_20230606T030349.nc
downloading S5P_OFFL_L2__O3__PR_20230605T083918_20230605T102048_29240_03_020500_20230607T005212.nc
downloading S5P_OFFL_L2__O3__PR_20230605T102048_20230605T120218_29241_03_020500_20230607T023628.nc
downloading S5P_OFFL_L2__O3__PR_20230606T082017_20230606T100147_29254_03_020500_20230608T003437.nc
downloadin

# Regrid TROPOMI to WRF grid

In [25]:
regrid_tropomi_on_wrf_grid_in_batch(meta_df, config.grid_id, config.tropomi_key)

Error: Environment variable 'ClimPy' is not set. Assume Workstation /home/osipovs/PycharmProjects/ClimPy/examples/regridding/regrid_tropomi_on_wrf_grid.py

0, Processing S5P_OFFL_L2__O3__PR_20230601T081351_20230601T095521_29183_03_020500_20230603T044544.nc
Will regrid this TROPOMI onto this WRF:
in grid /project/k10048/osipovs//Data/Copernicus/Sentinel-5P//S5P_OFFL_L2__O3__PR_20230601T081351_20230601T095521_29183_03_020500_20230603T044544.nc
out grid /project/k10048/osipovs//Data/Copernicus/Sentinel-5P//THOFA_d02//geo_em.nc
Lon[0,0]: 44.4158935546875, Lon[-1,-1]: 56.8514404296875
Lat[0,0]: 23.402915954589844, Lat[-1,-1]: 33.72188949584961


/home/osipovs/apps/mambaforge/envs/py310/lib/python3.10/site-packages/xarray/namedarray/core.py:264: UserWarning: Duplicate dimension names present: dimensions {'level'} appear more than once in dims=('time', 'scanline', 'ground_pixel', 'level', 'level'). We do not yet support duplicate dimension names, but we do allow initial construction of the object. We recommend you rename the dims immediately to become distinct, as most xarray functionality is likely to fail silently if you do not. To rename the dimensions you will need to set the ``.dims`` attribute of each variable, ``e.g. var.dims=('x0', 'x1')``.
  self._dims = self._parse_dimensions(dims)
/home/osipovs/apps/mambaforge/envs/py310/lib/python3.10/site-packages/xarray/namedarray/core.py:264: UserWarning: Duplicate dimension names present: dimensions {'level'} appear more than once in dims=('time', 'scanline', 'ground_pixel', 'level', 'level'). We do not yet support duplicate dimension names, but we do allow initial construction o

Processing O3 Profile switch


/home/osipovs/apps/mambaforge/envs/py310/lib/python3.10/site-packages/xesmf/smm.py:131: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')
/home/osipovs/apps/mambaforge/envs/py310/lib/python3.10/site-packages/xesmf/smm.py:131: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


ValueError: broadcasting cannot handle duplicate dimensions on a variable: ['scanline', 'ground_pixel', 'level', 'level']

# Derive List of TROPOMI files with good coverage

In [4]:
derive_information_fraction(meta_df, config.tropomi_key, config.grid_id)

# Derive TROPOMI like diagnistics for WRF output

In [5]:
for row in meta_df.itertuples():
    wrf_in = config.wrf_output_folder_path + row.wrfin_file
    wrf_out = config.wrf_output_folder_path + 'pp/tropomi_like_{}/'.format(config.diag_key.lower()) + row.wrfout_file
    tropomi_in=SENTINEL_DATA_ROOT_PATH + '/{}/'.format(config.grid_id) + row.Name

    if os.path.exists(wrf_out):
        print(f"  -> Skipping: Output already exists at {wrf_out}")
        continue

    # %run /home/osipovs/PycharmProjects/ClimPy/climpy/wrf/pp_wrf_like_tropomi_ch4.py --wrf_in={wrf_in} --wrf_out={wrf_out} --tropomi_in={tropomi_in}

    # Pure python version
    args = Namespace(
        wrf_in=wrf_in,
        wrf_out=wrf_out,
        tropomi_in=tropomi_in,
    )

    processors = {
        'ch4': pp_wrf_like_tropomi_ch4,
        'no2': pp_wrf_like_tropomi_no2,
        'so2': pp_wrf_like_tropomi_so2,
    }
    pp_wrf_like_tropomi_impl = processors[config.diag_key.lower()]
    pp_wrf_like_tropomi_impl(args)

  -> Skipping: Output already exists at /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_so2/wrfout_d01_2023-06-01_08_13_51
Will process this WRF:
in /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/wrfout_d01_2023-06-01_00_00_00
out /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_so2/wrfout_d01_2023-06-01_09_55_21
Remember that interpolated DIAG profile will contain NaNs if TROPOMI top is above WRF top
Remember that interpolated DIAG profile will contain NaNs if TROPOMI top is above WRF top
Saving to:
/scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_so2/wrfout_d01_2023-06-01_09_55_21
Done
Will process this WRF:
in /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/wrfout_d01_2023-06-02_00_00_00
out /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_so2/wrfout_d01_2023-06-02_07_54_50
Remember that interpolated DIAG profile will contain N